In [135]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd

In [136]:
data = pd.read_csv('./train.csv')
MAX_DEPTH = 500
MIN_NODE_LENGTH = 1
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.9,0.17,0.36,2.30,0.036,40.0,117.0,0.99340,3.07,1.10,10.5,5
1,6.5,0.21,0.34,13.00,0.053,55.0,177.0,0.99830,3.32,0.59,10.9,5
2,6.3,0.19,0.24,2.70,0.052,36.0,135.0,0.99374,3.60,0.81,11.9,6
3,5.6,0.17,0.27,2.70,0.030,31.0,103.0,0.98892,3.15,0.52,14.2,6
4,6.6,0.24,0.39,2.45,0.040,40.0,89.0,0.99110,3.15,0.55,12.5,5


In [137]:
def MSE(x):
    if len(x) == 0:
        return 0
    x_mean = np.mean(x)
    mse = np.square(x-x_mean)
    return np.sum(mse)

In [138]:
def absoluteError(x):
    if len(x) == 0:
        return 0
    x_median = np.median(x)
    sum = 0
    for i in x:
        sum += np.abs(i - x_median)
    return sum

In [139]:
errorFunctions = {'mse':MSE, 'absolute':absoluteError}

In [140]:
def normalize(mat):
    a = np.array(mat)
    a = a.astype(np.float64)
    b = np.apply_along_axis(lambda x: (x - np.min(x)) /
                            float(np.max(x) - np.min(x)), 0, a)
    return b

In [161]:
data = np.array(data).astype(np.float64)
np.random.shuffle(data)

split = int(0.9 * data.shape[0])

X = data[:split,:-1]
y = data[:split,-1]
y = y.reshape(-1,1)
data_val = data[split:,:]
leaves = []

X = normalize(X)


In [142]:
def splitValueSelection(a, Y, errorMetric = 'mse'):
    
    if len(a) == 1:
        print('length is 1')
    #select error function
    error_func = errorFunctions[errorMetric]
    parentError = error_func(Y)
    a = np.copy(a)
    Y = np.copy(Y)
    #sort a,Y according to a
    sort_a = a.argsort()
    a = a[sort_a]
    Y = Y[sort_a]
    
    #init max vals
    max_t = a[0]
    maxGain  = -np.inf
    
    # iterate over var for
    i = 0

    while i < len(a):
        while i<len(a)-1 and a[i]==a[i+1]:
            i += 1
        
        #leftList = a[:i+1]
        #rightList = a[i+1:]
        leftY = Y[:i]
        rightY = Y[i:]
        
        leftError = error_func(leftY)
        rightError = error_func(rightY)
        
        infGain = parentError - ( leftError + rightError)
        
        if a[i] == a[0]:
            infGain = -999999
            
        if infGain >= maxGain:
            maxGain = infGain
            max_t = a[i]
        elif maxGain == -np.inf:
            print(infGain,'infinity', maxGain)

        i += 1
    
    if maxGain == -np.inf:
            print('YOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
    
    return max_t, maxGain
    return None, None

In [143]:
def Entropy(x):
    unique, counts = np.unique(x, return_counts=True)
    relativeFrequency = counts/sum(counts)
    entropy = -sum(np.multiply(relativeFrequency, np.log2(relativeFrequency)))
    return entropy

In [144]:
def bestAttributeSplit(X,y):
    maxInfGain = -np.inf
    good_split = False
    index = None
    for i in range(X.shape[1]):
        infGain = splitValueSelection(X[:,i], y)[1]
        if infGain == None:
            continue
        if infGain > maxInfGain:
            good_split = True
            maxInfGain = infGain
            index = i
    if not good_split:
        return -1
    return index

In [145]:
class Node:
    
    def __init__(self, depth, X, y, data_val):
        self.X = np.copy(X)
        self.attributeIndex = None
        self.y = np.copy(y)
        self.depth = depth
        self.y_mean = np.mean(self.y)
        #self.threshold = 1
        self.data_val = np.copy(data_val)
        self.is_leaf = False
        if len(y) == 1:
            self.is_leaf == True
        else:
            self.assignAttr()
        
        
        
    def assignChildren(self, child_left, child_right):
        child_left.parent = self
        child_right.parent = self
        self.leftChild = child_left
        self.rightChild = child_right

    def assignAttr(self):
        self.attributeIndex = bestAttributeSplit(self.X, self.y)
        if self.attributeIndex == -1:
            self.is_leaf = True
            self.threshold = None
            self.infGain = None
        else:
            self.threshold, self.infGain = splitValueSelection(self.X[:,self.attributeIndex], self.y)    
            
            
            
    def check_with_threshold(self, x):
        #print(self.attributeIndex, x)
        return x[self.attributeIndex] >= self.threshold
           
    def splitValidationData(self):
        self.rightData = np.array([])
        self.leftData = np.array([])
        for i in range(self.data_val.shape[0]):
            if self.check_with_threshold(data_val[i]):
                np.append(self.rightData,data_val[i])
            else:
                np.append(self.leftData,data_val[i])
        if self.leftData.ndim == 1:
            self.leftData = self.leftData.reshape(-1,1)
        if self.rightData.ndim == 1:
            self.rightData = self.rightData.reshape(-1,1)
        
    def checkLeaf(self):
        if self.depth >= MAX_DEPTH or len(self.y) <= MIN_NODE_LENGTH or len(np.unique(self.y)) == 1 or self.is_leaf or MSE(y) < 0.2:
            self.is_leaf = True
            leaves.append(self)
        return self.is_leaf

In [146]:
def makeDecisionTree(node):
    if node.checkLeaf():
        #print(node.parent.y)
        print(node.y_mean, node.parent.attributeIndex, node.depth)
        return
    
    X = np.copy(node.X)
    y = np.copy(node.y)
    bestAttribute = node.attributeIndex
    if bestAttribute == -1:
        print('LOOOOOOOOOOOOOl')
        return
    sort_X = X[:,bestAttribute].argsort()
    X = X[sort_X]
    y = y[sort_X]
    split_index = np.searchsorted(X[:,bestAttribute], node.threshold, side = 'right') - 1
    
    #print('TEST', node.threshold in X[:,bestAttribute], node.threshold)
    
    X1, X2 = X[:split_index,:], X[split_index:,:]
    y1, y2 = y[:split_index], y[split_index:]
    #print(y1,y2)
    #print('splitIndex', split_index)
    
    
    
    if len(y1) > 0 and len(y1) < len(y):
        
        node.splitValidationData()
        
        node1 = Node(node.depth+1, X1, y1, node.leftData)
        node2 = Node(node.depth+1, X2, y2, node.rightData)
    
        node.assignChildren(node1, node2)

        makeDecisionTree(node1)
        makeDecisionTree(node2)
    else:
        print('here')
        node.is_leaf = True
        makeDecisionTree(node)
    return

In [147]:
leaves = []
rootNode = Node(0, X, y, data_val)
rootNode.parent = rootNode
makeDecisionTree(rootNode)

5.0 4 9
6.0 4 9
6.0 3 8
5.0 0 9
6.0 0 9
3.0 0 9
4.0 1 10
5.0 1 10
8.0 1 6
4.0 2 8
6.0 4 9
5.0 4 9
7.0 0 9
9.0 0 9
7.0 5 11
6.0 5 11
6.0 1 11
5.0 1 13
4.0 1 13
5.0 7 13
6.0 1 14
5.0 1 14
6.0 1 13
7.0 1 13
7.0 0 13
8.0 0 13
6.0 1 13
5.0 1 13
6.0 9 14
6.0 5 15
7.0 5 15
6.0 1 14
7.0 1 14
6.0 2 12
5.0 2 12
5.0 3 11
5.0 5 11
6.0 2 12
5.0 1 13
6.0 1 13
7.0 7 12
6.0 7 12
5.0 3 11
5.0 1 10
6.0 1 10
6.0 2 9
8.0 0 10
7.0 0 10
5.0 3 12
6.0 3 12
6.0 0 11
5.0 10 10
7.0 6 9
5.0 7 12
6.0 7 12
5.0 0 12
6.0 0 12
6.0 3 10
5.0 4 10
4.0 4 10
6.0 4 10
5.0 4 10
6.0 8 10
8.0 0 11
7.0 0 12
7.0 1 13
6.0 1 13
3.0 0 9
5.0 0 9
8.0 8 11
7.0 8 11
7.0 7 11
8.0 7 11
7.0 1 10
9.0 2 11
8.0 2 11
7.0 0 9
5.0 0 10
6.0 0 10
8.0 1 10
7.0 8 12
6.0 8 12
8.0 0 11
8.0 8 11
7.0 1 12
6.0 4 13
7.0 4 13
6.0 0 13
7.0 0 13
7.0 1 15
6.0 1 15
7.0 2 15
6.0 2 15
7.0 0 16
6.0 0 16
7.0 1 15
7.0 4 14
7.0 0 13
6.0 0 14
7.0 0 14
6.0 3 12
7.0 4 12
6.0 4 12
8.0 2 11
7.0 4 13
6.0 4 13
6.0 5 13
7.0 5 13
6.0 1 12
5.0 0 13
6.0 0 13
6.0 9 11
3.0 2 13

6.0 8 15
5.0 1 16
6.0 1 16
7.0 3 14
5.0 0 15
6.0 0 15
4.0 4 14
7.0 0 13
5.0 0 13
7.0 0 16
6.0 0 16
6.0 1 15
7.0 4 16
6.0 4 16
7.0 0 16
6.0 0 16
5.0 9 13
5.0 6 12
8.0 1 12
7.0 1 12
8.0 5 10
7.0 0 11
6.0 0 11
5.0 2 11
4.0 2 11
7.0 3 8
6.0 9 11
5.0 9 11
5.0 3 10
6.0 2 12
5.0 2 12
6.0 1 11
6.0 0 11
5.0 0 11
6.0 0 12
5.0 0 12
6.0 9 11
5.0 5 10
3.0 5 12
4.0 0 13
5.0 0 13
5.0 10 11
6.0 3 14
5.0 3 14
5.0 5 13
3.0 0 13
5.0 0 13
6.0 1 11
5.0 5 11
6.0 5 12
7.0 5 12
8.0 0 11
7.0 0 11
5.0 1 11
6.0 1 11
6.0 2 17
6.0 6 19
5.0 6 19
6.0 4 18
5.0 6 17
5.0 0 18
4.0 0 18
6.0 1 15
4.0 9 14
6.0 1 14
5.0 1 14
6.0 0 12
5.0 1 12
6.0 0 13
7.0 0 13
8.0 1 8
5.0 1 8
5.0 1 14
6.0 3 15
5.0 3 15
6.0 4 13
6.0 0 12
4.0 4 11
4.0 6 11
5.0 0 12
4.0 0 12
6.0 4 12
5.0 4 12
5.0 3 11
5.0 0 14
6.0 0 14
5.0 2 13
6.0 5 12
6.0 1 12
5.0 1 12
5.0 3 9
6.0 3 9
5.0 2 9
6.0 2 9
5.0 3 9
6.0 3 9
6.0 10 8
5.0 4 9
6.0 4 9
5.0 6 8
4.0 6 8
4.0 0 9
5.0 0 9
3.0 0 8
3.0 1 6
6.0 1 5


In [148]:
print('Decision Tree created')
print('No. of leaves =', len(leaves))

Decision Tree created
No. of leaves = 1093


In [149]:
def prune_condition(node, error_metric='mse'):
    parent = node
    error_func = errorFunctions[error_metric]
    parentError = error_func(parent.data_val[:,-1])
    leftError = error_func(parent.leftChild.data_val[:,-1])
    rightError = error_func(parent.rightChild.data_val[:,-1])
    if parent.data_val.shape[0] == 0:
        infGain = 0
    else:
        print('yooo')
        infGain = parentError\
                - (parent.leftChild.data_val.shape[0] * leftError \
                   + (parent.rightChild.data_val.shape[0]) * rightError)/parent.data_val.shape[0]
    #print(infGain)
    return infGain


In [150]:
def error(y1, x):
    if len(y1) == 0:
        return 0
    if x.ndim > 1:
        y2 = np.array([predict(i) for i in x])
    else:
        y2 = predict(x)
    sum_error = 0
    for i in range(len(y1)):
        sum_error += (y1[i]-y2[i])**2
    return sum_error/len(y1)

In [151]:
def prune(leaves, count):
    if len(leaves) < 2:
        print(count)
        return count
    parents = []
    for leaf in leaves:
        leaf = leaf.parent
        if leaf.depth == 1:
            print(count)
            return count

        parent = leaf.parent
        l = parent.leftChild
        r = parent.rightChild
        if error(parent.data_val[:,:-1],parent.data_val[:,-1]) <= (error(l.data_val[:,:-1],l.data_val[:,-1]) + error(r.data_val[:,:-1],r.data_val[:,-1])):
            parent.is_leaf = True
            if parent not in parents:
                count += 1
                parents.append(leaf.parent)
    return prune(parents, count)


In [152]:
def predict(x):
    node = rootNode
    while True:
        if node.is_leaf:
            return node.y_mean
        if (node.check_with_threshold(x)):
            node = node.rightChild
        elif (not node.check_with_threshold(x)):
            node = node.leftChild
        else:
            print('ERROR', node.nChild)
            

In [153]:
def checkTree(n):
    x = X[n]
    ans = y[n]
    print(predict(x), ans)

In [154]:
print(error(y,X))
checkTree(np.random.randint(0,X.shape[0]))


[0.44910019]
6.0 [6.]


In [155]:
p = prune(leaves, 0)
print(p)


945
945


In [156]:
X_test = np.array(pd.read_csv('./test.csv'))
X_test = normalize(X_test)

In [157]:
print(error(y,X))

[0.74861181]


In [158]:
checkTree(np.random.randint(0,X.shape[0]))

5.68463476070529 [5.]


In [159]:
if X.shape[1] == 11:
    key = 'quality'
else:
    key = 'output'
f = open('./output.csv','w')
f.write('Id,'+key+'\n')
for i in range(X_test.shape[0]):
    f.write(str(i+1)+','+str(predict(X_test[i]))+'\n')
f.close()

In [160]:
#leaves.sort(key = lambda x: x.y_mean)

In [134]:
np.searchsorted([1,2,2,2,3,4,5], 2, side = 'right')

4